# Métaheuristique pour le problème quadratique binaire non constraint

## Définition du problème

Soit $Q \in \mathbb{Z}^{n\times n}$ une matrice symétrique.
Soit $x_1,\dots, x_n$ des variables binaires, c'est-à-dire pouvant prendre la valeur 0 ou 1.

Le problème d'optimisation quadratique binaire non contraint consiste à déterminer la valeur des variables $x_j$, $j=1, \dots, n$ maximisant :

$$
\sum_{i = \in \{1,\dots,n\}} \sum_{j \in \{1,\dots,n\}:i\neq j} Q_{ij} x_i x_j
$$

On cherche à implémenter une métaheuristique pour résoudre ce problème.

In [1]:
import random as r
import math as m

In [2]:
def create_random_instance(n):
    d = [ [r.randint(-100, 100) for _ in range(n)] for _ in range(n) ]
    
    for i in range(n):
        for j in range(i + 1, n):
            d[j][i] = d[i][j]
        d[i][i] = 0
    return d

def afficher_instance(d):
    n = len(d)
    for i in range(n):
        print(str(d[i]).replace(",", "\t"))


ins = create_random_instance(10)
afficher_instance(ins)

[0	 -32	 92	 -5	 -67	 -69	 89	 -61	 -51	 59]
[-32	 0	 10	 -38	 11	 98	 -69	 -51	 87	 -38]
[92	 10	 0	 -85	 42	 -27	 76	 47	 47	 -26]
[-5	 -38	 -85	 0	 -60	 26	 -63	 18	 37	 15]
[-67	 11	 42	 -60	 0	 96	 44	 73	 80	 -68]
[-69	 98	 -27	 26	 96	 0	 57	 -22	 -5	 -91]
[89	 -69	 76	 -63	 44	 57	 0	 56	 74	 -37]
[-61	 -51	 47	 18	 73	 -22	 56	 0	 12	 39]
[-51	 87	 47	 37	 80	 -5	 74	 12	 0	 26]
[59	 -38	 -26	 15	 -68	 -91	 -37	 39	 26	 0]


In [3]:
def random_solution(ins):
    return [ r.randint(0, 1) for _ in range(len(ins))]

sol = random_solution(ins)
print(sol)

[0, 0, 0, 0, 1, 0, 0, 1, 1, 0]


In [4]:
def cost_sol(ins, sol):
    n = len(sol)
    total = 0
    for i in range(n):
        for j in range(n):
            total += ins[i][j] * sol[i] * sol[j]
    return total

print(cost_sol(ins, sol))

330


## Métaheuristique de descente

In [5]:
def descente(ins, initial_sol, find_best_neighbor):
    """
    Effectue une métaheuristique de type descente.
    Paramètres : 
    - ins: instance du problème
    - initial_sol: solution initial
    - find_best_neighbor: fonction cherchant la meilleure solution voisine et 
                          mettant à jour la solution passée en paramètre.
                          -> find_best_neighbor(ins, s) retourne True si s a été modifiée 
    """
    s = initial_sol.copy()
    hasFoundBetterSol = True
    while hasFoundBetterSol:
        hasFoundBetterSol = find_best_neighbor(ins, s)
    return s


def find_best_1change(ins, s):
    best = 0
    best_ind = -1
    for i in range(len(s)):
        
        diff = 0
        for j in range(len(s)):
            diff += ins[i][j] * s[j]
        if s[i] == 1 and -diff > best: 
            best = -diff
            best_ind = i
        elif s[i] == 0 and diff > best:
            best = diff
            best_ind = i
    if best > 0:
        s[best_ind] = 1 if s[best_ind] == 0 else 0
        return True
    else:
        return False

In [9]:
def descente_multistart(ins, find_best_neighbor, n_tentatives):
    best_solution = random_solution(ins) 
    for i in range(n_tentatives):
        solution = random_solution(ins)
        best_locale = descente(ins, solution, find_best_neighbor)
        if cost_sol(ins, best_locale) > cost_sol(ins, best_solution):
            best_solution = best_locale
    return best_solution
        
    

In [19]:
ins = create_random_instance(200)

sol_initiale = random_solution(ins)
sol_meta = descente(ins, sol_initiale, find_best_1change)
print("La métaheuristique de descente trouve une solution de cout", cost_sol(ins, sol_meta))


sol_multistart = descente_multistart(ins, find_best_1change, 20)
print("La métaheuristique de descente  Multi-start (20) trouve une solution de cout", 
      cost_sol(ins, sol_multistart))



La métaheuristique de descente trouve une solution de cout 90182
La métaheuristique de descente  Multi-start (20) trouve une solution de cout 92800


## Recuit simulé

In [34]:
def one_neighbor(ins, sol, T):
    # On teste la solution où l'on permute la variable x_i
    i = r.randint(0, len(sol)-1)
    
    # Calcul de la difference de coût delta(s, s')
    diff = 0
    for j in range(len(sol)):
        diff += ins[i][j] * sol[j]
 
    if sol[i] == 1 : 
        diff = -diff
    
    nb = r.random()
   
    if T <= 0.0001 or (-diff/T) >= 1 or nb < m.exp(-diff/T):
        sol[i] = 1 if sol[i] == 0 else 0
    
    
    
        
def recuit_simule(ins, solution_initiale, T0, alpha, max_tentatives, get_one_neighbor):
    T = T0
    s = solution_initiale.copy()
    for _ in range(max_tentatives):
        one_neighbor(ins, s, T)
        T = alpha * T
    return s
        
ins = create_random_instance(200)
sol_initiale = random_solution(ins)
sol_recuit = recuit_simule(ins, sol_initiale, 10000, 0.99, 1000000, one_neighbor)
print("Le recuit simulé trouve une solution de cout", cost_sol(ins, sol_recuit))
sol_meta = descente(ins, sol_recuit, find_best_1change)
print("Recuit simulé + descente trouve une soluiton de cout", cost_sol(ins, sol_meta))


sol_meta = descente(ins, sol_initiale, find_best_1change)
print("Descente seule trouve une solution de cout", cost_sol(ins, sol_meta))


Le recuit simulé trouve une solution de cout 12970
Recuit simulé + descente trouve une soluiton de cout 103358
Descente seule trouve une solution de cout 101066
